# **JOWMDroid**

Método de detecção de aplicativos maliciosos Android com base na ponderação de recursos com otimização conjunta de mapeamento de peso e parâmetros de classificação.

## Importações

In [41]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression

# métricas
from sklearn.model_selection import cross_val_score

# modelos
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [42]:
# tirando limitações de exibição de linhas e colunas do dataset
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Datasets

Drebin

In [43]:
drebin = pd.read_csv("Drebin215.csv", sep=";")

In [44]:
drebin['class'].replace('B', 0, inplace=True)
drebin['class'].replace('S', 1, inplace=True)

In [45]:
drebin.head()

,transact,onServiceConnected,bindService,attachInterface,ServiceConnection,android.os.Binder,android.permission.SEND_SMS,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.getMethods,Ljava.lang.Class.cast,Ljava.net.URLDecoder,android.content.pm.Signature,android.telephony.SmsManager,android.permission.READ_PHONE_STATE,getBinder,ClassLoader,Landroid.content.Context.registerReceiver,Ljava.lang.Class.getField,Landroid.content.Context.unregisterReceiver,android.permission.GET_ACCOUNTS,android.permission.RECEIVE_SMS,Ljava.lang.Class.getDeclaredField,android.permission.READ_SMS,getCallingUid,Ljavax.crypto.spec.SecretKeySpec,android.intent.action.BOOT_COMPLETED,android.permission.USE_CREDENTIALS,android.permission.MANAGE_ACCOUNTS,android.content.pm.PackageInfo,KeySpec,TelephonyManager.getLine1Number,DexClassLoader,HttpGet.init,SecretKey,Ljava.lang.Class.getMethod,System.loadLibrary,android.intent.action.SEND,Ljavax.crypto.Cipher,android.permission.WRITE_SMS,android.permission.READ_SYNC_SETTINGS,android.permission.AUTHENTICATE_ACCOUNTS,android.telephony.gsm.SmsManager,com.android.browser.permission.WRITE_HISTORY_BOOKMARKS,TelephonyManager.getSubscriberId,mount,android.permission.INSTALL_PACKAGES,Runtime.getRuntime,android.permission.CAMERA,Ljava.lang.Object.getClass,android.permission.WRITE_SYNC_SETTINGS,com.android.browser.permission.READ_HISTORY_BOOKMARKS,Ljava.lang.Class.forName,android.permission.INTERNET,android.intent.action.PACKAGE_REPLACED,Binder,android.intent.action.SEND_MULTIPLE,android.permission.RECORD_AUDIO,IBinder,android.os.IBinder,createSubprocess,android.permission.NFC,android.permission.ACCESS_LOCATION_EXTRA_COMMANDS,URLClassLoader,android.permission.WRITE_APN_SETTINGS,abortBroadcast,android.permission.BIND_REMOTEVIEWS,android.intent.action.TIME_SET,android.permission.READ_PROFILE,TelephonyManager.getDeviceId,android.permission.MODIFY_AUDIO_SETTINGS,getCallingPid,android.permission.READ_SYNC_STATS,android.permission.BROADCAST_STICKY,android.intent.action.PACKAGE_REMOVED,android.intent.action.TIMEZONE_CHANGED,android.permission.WAKE_LOCK,android.permission.RECEIVE_BOOT_COMPLETED,android.permission.RESTART_PACKAGES,Ljava.lang.Class.getPackage,chmod,Ljava.lang.Class.getDeclaredClasses,android.intent.action.ACTION_POWER_DISCONNECTED,android.intent.action.PACKAGE_ADDED,PathClassLoader,TelephonyManager.getSimSerialNumber,Runtime.load,TelephonyManager.getCallState,android.permission.BLUETOOTH,android.permission.READ_CALENDAR,android.permission.READ_CALL_LOG,android.permission.SUBSCRIBED_FEEDS_WRITE,android.permission.READ_EXTERNAL_STORAGE,TelephonyManager.getSimCountryIso,sendMultipartTextMessage,PackageInstaller,android.permission.VIBRATE,remount,android.intent.action.ACTION_SHUTDOWN,sendDataMessage,android.permission.ACCESS_NETWORK_STATE,chown,HttpPost.init,Ljava.lang.Class.getClasses,android.permission.SUBSCRIBED_FEEDS_READ,TelephonyManager.isNetworkRoaming,android.permission.CHANGE_WIFI_MULTICAST_STATE,android.permission.WRITE_CALENDAR,android.intent.action.PACKAGE_DATA_CLEARED,android.permission.MASTER_CLEAR,HttpUriRequest,android.permission.UPDATE_DEVICE_STATS,android.permission.WRITE_CALL_LOG,android.permission.DELETE_PACKAGES,android.permission.GET_TASKS,android.permission.GLOBAL_SEARCH,android.permission.DELETE_CACHE_FILES,android.permission.WRITE_USER_DICTIONARY,android.intent.action.PACKAGE_CHANGED,android.intent.action.NEW_OUTGOING_CALL,android.permission.REORDER_TASKS,android.permission.WRITE_PROFILE,android.permission.SET_WALLPAPER,android.permission.BIND_INPUT_METHOD,divideMessage,android.permission.READ_SOCIAL_STREAM,android.permission.READ_USER_DICTIONARY,android.permission.PROCESS_OUTGOING_CALLS,android.permission.CALL_PRIVILEGED,Runtime.exec,android.permission.BIND_WALLPAPER,android.permission.RECEIVE_WAP_PUSH,android.permission.DUMP,android.permission.BATTERY_STATS,android.permission.ACCESS_COARSE_LOCATION,android.permission.SET_TIME,android.intent.action.SENDTO,android.permission.WRITE_SOCIAL_STREAM,an

## Mutual Information (IG)

Utilizamos o mutual_info_regression como método de Ganho de Informação (IG) para selecionar as características mais relevantes para o modelo. A nota de corte é de 0.05.

In [46]:
x_aux = drebin.drop(columns=["class"])
y_aux = drebin["class"]

In [47]:
def calculate_mi(x, y):
    columns = x.columns
    mi = mutual_info_regression(x, y)
    return pd.Series(mi, index=np.array(columns)).sort_values(ascending=False)

In [48]:
mi = calculate_mi(x_aux, y_aux)
mi

transact                                                     0.193059
onServiceConnected                                           0.191269
attachInterface                                              0.177529
bindService                                                  0.175022
ServiceConnection                                            0.173914
android.os.Binder                                            0.171117
android.permission.SEND_SMS                                  0.143469
Ljava.lang.Class.getCanonicalName                            0.139168
Ljava.lang.Class.getMethods                                  0.118408
Ljava.lang.Class.cast                                        0.105963
Ljava.net.URLDecoder                                         0.105665
android.content.pm.Signature                                 0.104013
getBinder                                                    0.097407
android.telephony.SmsManager                                 0.094700
android.permission.R

In [49]:
drebin_mi = pd.DataFrame()
drebin_mi = drebin[["transact",
"attachInterface",
"onServiceConnected",
"bindService",
"android.os.Binder",
"ServiceConnection",
"android.permission.SEND_SMS",
"Ljava.lang.Class.getCanonicalName",
"Ljava.lang.Class.cast",
"Ljava.lang.Class.getMethods",
"Ljava.net.URLDecoder",
"android.content.pm.Signature",
"android.telephony.SmsManager",
"android.permission.READ_PHONE_STATE",
"ClassLoader",
"getBinder",
"Ljava.lang.Class.getField",
"android.permission.READ_SMS",
"android.permission.RECEIVE_SMS",
"Landroid.content.Context.unregisterReceiver",
"Landroid.content.Context.registerReceiver",
"android.permission.GET_ACCOUNTS",
"Ljava.lang.Class.getDeclaredField",
"getCallingUid",
"HttpGet.init",
"Ljava.lang.Class.getMethod",
"Ljavax.crypto.spec.SecretKeySpec",
"android.intent.action.BOOT_COMPLETED",
"class"]]

In [50]:
drebin_mi.shape

(15036, 29)

## Separando conjunto de dados

Variáveis alvo e preditoras

In [51]:
x = drebin_mi.drop(columns=["class"])
y = drebin_mi["class"]

Variáveis de treino e teste

In [52]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)

## Modelos para mapear pesos

Utilizamos esses três modelos abaixo por causa de seus métodos de seleção de características, que através de seus cálculos, distribuem pesos para as características. Após cada modelo distribuir os pesos das características, normalizamos os pesos entre 0 e 1 para todos os modelos e em seguida tiramos a média.

In [53]:
# SVM
model_SVM = SVC(kernel='linear')

# Random Forest
model_RF = RandomForestClassifier()

# Logistic Regression
model_LR = LogisticRegression()

Treinamento

In [54]:
# SVM
model_SVM.fit(x_train, y_train)

# Random Forest
model_RF.fit(x_train, y_train)

# Logistic Regression
model_LR.fit(x_train, y_train)

LogisticRegression()

### Mapeando pesos iniciais

In [55]:
# SVM
SVM_weights = model_SVM.coef_

# RandomForest
RF_weights = model_RF.feature_importances_

# Logistic Regression
LR_weights = model_LR.coef_

Maiores e menores valores

In [56]:
# SVM
SVM_max = SVM_weights.max()
SVM_min = SVM_weights.min()

# RandomForest
RF_max = RF_weights.max()
RF_min = RF_weights.min()

# Logistic Regression
LR_max = LR_weights.max()
LR_min = LR_weights.min()

Função para normalizar os dados entre 0 e 1

In [57]:
# Vetores que irão contem os pesos normalizados das características
SVM_array = []
RF_array = []
LR_array = []

# FUNÇÕES DE NORMALIZAÇÃO DE PESOS
# SVM
for x in SVM_weights:
    if SVM_max != SVM_min:
        result_SVM = (x - SVM_min) / (SVM_max - SVM_min)
        SVM_array.append(result_SVM)
    else:
        result_SVM = 0.5
        SVM_array.append(result_SVM)

# RandomForest
for x in RF_weights:
    if RF_max != RF_min:
        result_RF = (x - RF_min) / (RF_max - RF_min)
        RF_array.append(result_RF)
    else:
        result_RF = 0.5
        RF_array.append(result_RF)

# LR
for x in LR_weights:
    if LR_max != LR_min:
        result_LR = (x - LR_min) / (LR_max - LR_min)
        LR_array.append(result_LR)
    else:
        result_LR = 0.5
        LR_array.append(result_LR)

Média dos pesos dos três modelos

In [58]:
init_weights = (SVM_array[0] + RF_array[0] + LR_array[0]) / 3

In [59]:
init_weights

array([0.33502459, 0.45244476, 0.37912771, 0.58057366, 0.47195802,
       0.53553678, 0.84671564, 0.37264824, 0.50691908, 0.38713601,
       0.41996396, 0.52046831, 0.70070952, 0.86011439, 0.49148327,
       0.30176435, 0.48228463, 0.63857318, 0.42457567, 0.4122956 ,
       0.41751016, 0.32864698, 0.60248851, 0.21112121, 0.56472498,
       0.57844865, 0.51173758, 0.64773062])

## Funções de Mapeamento de Pesos

Utilizamos 5 funções de mapeamento de pesos: Potência, Exponencial, Logarítmica, Hiperbólica e Curva em S.

In [60]:
init_weights.max()

0.8601143949405626

In [61]:
def power_function(v):
    a, y, x = v
    result = y * pow(x, exp = a)
    return result

def exponential_function(v):
    a, b, y, x = v
    result = y * (pow(a, exp=(b*x)) - 1) / (pow(a, exp=b) - 1)
    return result

def logarithmic_function(v):
    a, y, x = v
    result = y * ((np.log(1 + a*x)) / (np.log(1 + a)))
    return result

def hyperbolic_function(v):
    a, b, y, x = v
    result = (y * ((a*x) / (1 + (b*x)))) / (a / (1 + b))
    return result

def S_curve_function(v):
    a, y, x = v
    result = (y * ((1 / (1 + (a * np.exp(-x)))) - (1 / (1 + a)))) / ((1 / (1 + (a*np.exp(-1)))) - (1 / (1+a)))
    return result

### LR Parametrizado

#### Testando o modelo LR

In [62]:
power_weights_LR = []
exponential_weights_LR = []
logarithmic_weights_LR = []
hyperbolic_weights_LR = []
S_curve_weights_LR = []

for w in init_weights:
    power_w = power_function([0.070, 2.190, w])
    power_weights_LR.append(power_w)

    exponential_w = exponential_function([2.108, 1.107, 2.963, w])
    exponential_weights_LR.append(exponential_w)

    logarithmic_w = logarithmic_function([2.523, 2.427, w])
    logarithmic_weights_LR.append(logarithmic_w)

    hyperbolic_w = hyperbolic_function([8.205, 6.394, 1.656, w])
    hyperbolic_weights_LR.append(hyperbolic_w)

    S_curve_w = S_curve_function([2.959, 2.422, w])
    S_curve_weights_LR.append(S_curve_w)

#### Função para podar features que tenham peso maior que o peso máximo do init_weights

In [63]:
drebin_power = pd.DataFrame()
drebin_exponential = pd.DataFrame()
drebin_logarithmic = pd.DataFrame()
drebin_hyperbolic = pd.DataFrame()
drebin_S_curve = pd.DataFrame()

In [64]:
max = init_weights.max()
max

0.8601143949405626

#### Função de Potência

In [65]:
count = 0
count_aux_power = []
for weight in power_weights_LR:
    if weight >= 0 and weight <= max:
        count_aux_power.append(count)
    else:
        print('')
    count += 1

In [66]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_power:
        if count == x:
            drebin_power[col] = drebin[col]
            drebin_power.replace(1, power_weights_LR[x], inplace=True)
    count+=1

#### Função Exponencial

In [67]:
count = 0
count_aux_exponential = []
for weight in exponential_weights_LR:
    if weight >= 0 and weight <= max:
        count_aux_exponential.append(count)
    else:
        ''
    count += 1

In [68]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_exponential:
        if count == x:
            drebin_exponential[col] = drebin[col]
            drebin_exponential.replace(1, exponential_weights_LR[x], inplace=True)
    count+=1

#### Função Logarítmica

In [69]:
count = 0
count_aux_logarithmic = []
for weight in logarithmic_weights_LR:
    if weight >= 0 and weight <= max:
        count_aux_logarithmic.append(count)
    else:
        ''
    count += 1

In [70]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_logarithmic:
        if count == x:
            drebin_logarithmic[col] = drebin[col]
            drebin_logarithmic.replace(1, logarithmic_weights_LR[x], inplace=True)
    count+=1

#### Função Hiperbólica

In [71]:
count = 0
count_aux_hyperbolic = []
for weight in hyperbolic_weights_LR:
    if weight >= 0 and weight <= max:
        count_aux_hyperbolic.append(count)
    else:
        ''
    count += 1

In [72]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_hyperbolic:
        if count == x:
            drebin_hyperbolic[col] = drebin[col]
            drebin_hyperbolic.replace(1, hyperbolic_weights_LR[x], inplace=True)
    count+=1

#### Função Curva_S

In [73]:
count = 0
count_aux_S_curve = []
for weight in S_curve_weights_LR:
    if weight >= 0 and weight <= max:
        count_aux_S_curve.append(count)
    else:
        ''
    count += 1

In [74]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_S_curve:
        if count == x:
            drebin_S_curve[col] = drebin[col]
            drebin_S_curve.replace(1, S_curve_weights_LR[x], inplace=True)
    count+=1

#### Shapes

In [75]:
print("Drebin >>", drebin.shape)
print("\nDrebin_Power >>", drebin_power.shape)
print("Drebin_Exponential >>", drebin_exponential.shape)
print("Drebin_Logarithmic >>", drebin_logarithmic.shape)
print("Drebin_Hyperbolic >>", drebin_hyperbolic.shape)
print("Drebin_S_Curve >>", drebin_S_curve.shape)

Drebin >> (15036, 215)

Drebin_Power >> (0, 0)
Drebin_Exponential >> (15036, 6)
Drebin_Logarithmic >> (15036, 1)
Drebin_Hyperbolic >> (0, 0)
Drebin_S_Curve >> (15036, 7)


In [76]:
drebin_power['class'] = drebin['class']
drebin_exponential['class'] = drebin['class']
drebin_logarithmic['class'] = drebin['class']
drebin_hyperbolic['class'] = drebin['class']
drebin_S_curve['class'] = drebin['class']

#### Treinando modelos com features selecionadas

Variáveis preditoras e alvo

In [77]:
# Power
x_power = drebin_power.drop(columns=['class'])
y_power = drebin_power['class']

# Exponential
x_exponential = drebin_exponential.drop(columns=['class'])
y_exponential = drebin_exponential['class']

# Logarithmic
x_logarithmic = drebin_logarithmic.drop(columns=['class'])
y_logarithmic = drebin_logarithmic['class']

# Hyperbolic
x_hyperbolic = drebin_hyperbolic.drop(columns=['class'])
y_hyperbolic = drebin_hyperbolic['class']

# S_Curve
x_S_curve = drebin_S_curve.drop(columns=['class'])
y_S_curve = drebin_S_curve['class']

Variáveis de treino e teste

In [78]:
x_train_power, x_test_power, y_train_power, y_test_power = train_test_split(x_power, y_power, train_size=0.8)
x_train_exponential, x_test_exponential, y_train_exponential, y_test_exponential = train_test_split(x_exponential, y_exponential, train_size=0.8)
x_train_logarithmic, x_test_logarithmic, y_train_logarithmic, y_test_logarithmic = train_test_split(x_logarithmic, y_logarithmic, train_size=0.8)
x_train_hyperbolic, x_test_hyperbolic, y_train_hyperbolic, y_test_hyperbolic = train_test_split(x_hyperbolic, y_hyperbolic, train_size=0.8)
x_train_S_curve, x_test_S_curve, y_train_S_curve, y_test_S_curve = train_test_split(x_S_curve, y_S_curve, train_size=0.8)

#### Modelos SVM

In [79]:
model_LR_power = LogisticRegression()
model_LR_exponential = LogisticRegression()
model_LR_logarithmic = LogisticRegression()
model_LR_hyperbolic = LogisticRegression()
model_LR_S_curve = LogisticRegression()

Treinamento dos modelos

In [80]:
#model_LR_power.fit(x_train_power, y_train_power)
model_LR_exponential.fit(x_train_exponential, y_train_exponential)
model_LR_logarithmic.fit(x_train_logarithmic, y_train_logarithmic)
#model_LR_hyperbolic.fit(x_train_hyperbolic, y_train_hyperbolic)
model_LR_S_curve.fit(x_train_S_curve, y_train_S_curve)

LogisticRegression()

#### Métricas

In [81]:
# Power
#print("Acurácia >> ", (cross_val_score(model_LR_power, x_test_power, y_test_power, scoring="accuracy", cv = 5).mean())*100,"%")
#print("Precisão >> ", (cross_val_score(model_LR_power, x_test_power, y_test_power, scoring="precision", cv = 5).mean())*100,"%")
#print("Recall >> ", (cross_val_score(model_LR_power, x_test_power, y_test_power, scoring="recall", cv = 5).mean())*100,"%")
#print("F1-Score >> ", (cross_val_score(model_LR_power, x_test_power, y_test_power, scoring="f1", cv = 5).mean())*100,"%")

In [82]:
# Exponential
print("Acurácia >> ", (cross_val_score(model_LR_exponential, x_test_exponential, y_test_exponential, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_LR_exponential, x_test_exponential, y_test_exponential, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_LR_exponential, x_test_exponential, y_test_exponential, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_LR_exponential, x_test_exponential, y_test_exponential, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  76.76209639526593 %
Precisão >>  63.12387880704902 %
Recall >>  86.3302752293578 %
F1-Score >>  72.91586357008242 %


In [83]:
# Logarithmic
print("Acurácia >> ", (cross_val_score(model_LR_logarithmic, x_test_logarithmic, y_test_logarithmic, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_LR_logarithmic, x_test_logarithmic, y_test_logarithmic, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_LR_logarithmic, x_test_logarithmic, y_test_logarithmic, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_LR_logarithmic, x_test_logarithmic, y_test_logarithmic, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  62.50004145914063 %
Precisão >>  0.0 %
Recall >>  0.0 %
F1-Score >>  0.0 %


/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: Un

In [84]:
# Hyperbolic
#print("Acurácia >> ", (cross_val_score(model_LR_hyperbolic, x_test_hyperbolic, y_test_hyperbolic, scoring="accuracy", cv = 5).mean())*100,"%")
#print("Precisão >> ", (cross_val_score(model_LR_hyperbolic, x_test_hyperbolic, y_test_hyperbolic, scoring="precision", cv = 5).mean())*100,"%")
#print("Recall >> ", (cross_val_score(model_LR_hyperbolic, x_test_hyperbolic, y_test_hyperbolic, scoring="recall", cv = 5).mean())*100,"%")
#print("F1-Score >> ", (cross_val_score(model_LR_hyperbolic, x_test_hyperbolic, y_test_hyperbolic, scoring="f1", cv = 5).mean())*100,"%")

In [85]:
# S_curve
print("Acurácia >> ", (cross_val_score(model_LR_S_curve, x_test_S_curve, y_test_S_curve, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_LR_S_curve, x_test_S_curve, y_test_S_curve, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_LR_S_curve, x_test_S_curve, y_test_S_curve, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_LR_S_curve, x_test_S_curve, y_test_S_curve, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  78.65810581478267 %
Precisão >>  65.33053277187369 %
Recall >>  86.58516811742618 %
F1-Score >>  74.45879515441786 %


In [86]:
drebin_power.head()

,class
0,1
1,1
2,1
3,1
4,1


In [87]:
drebin_S_curve.head()

,transact,bindService,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.cast,ClassLoader,Ljava.lang.Class.getDeclaredField,getCallingUid,class
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,1
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,1
2,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,1
3,0.0,0.0,0.0,0.853654,0.000000,0.716092,0.0,1
4,0.0,0.0,0.0,0.000000,0.653822,0.716092,0.0,1


### Logistic Regression Original

In [88]:
# criando modelo
model_LR_original = LogisticRegression()

# treinando modelo
model_LR_original.fit(x_train, y_train)

# métricas modelo
print("Acurácia >> ", (cross_val_score(model_LR_original, x_test, y_test, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_LR_original, x_test, y_test, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_LR_original, x_test, y_test, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_LR_original, x_test, y_test, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  91.82138296637386 %
Precisão >>  89.69534904471985 %
Recall >>  88.91774891774892 %
F1-Score >>  89.29938263706998 %


## Algoritmo de Evolução Diferencial (ED)

Utilizamos o algoritmo ED para otimizar em conjunto os parâmetros da função de mapeamento de peso e do classificador final.

In [89]:
from scipy.optimize import differential_evolution

### Função de Potência

In [90]:
bounds = [(0.0, 10.0), (pow(10.0, exp=-6), 10.0), (0.0, 1.0)]
result = differential_evolution(power_function, bounds = bounds, popsize = 40, maxiter = 30, recombination = 0.3, disp = True, mutation=0.5, seed=1)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution_power = result['x']
evaluation = power_function(solution_power)
print('Solution_power: f(%s) = %.5f' % (solution_power, evaluation))
print(result.fun)

differential_evolution step 1: f(x)= 5.24883e-22
differential_evolution step 2: f(x)= 5.24883e-22
differential_evolution step 3: f(x)= 3.63957e-26
differential_evolution step 4: f(x)= 1.19724e-28
differential_evolution step 5: f(x)= 1.04443e-29
differential_evolution step 6: f(x)= 1.04443e-29
differential_evolution step 7: f(x)= 1.04443e-29
differential_evolution step 8: f(x)= 1.04443e-29
differential_evolution step 9: f(x)= 6.18501e-30
differential_evolution step 10: f(x)= 3.22298e-30
differential_evolution step 11: f(x)= 1.17696e-38
differential_evolution step 12: f(x)= 2.07954e-39
differential_evolution step 13: f(x)= 1.15057e-39
differential_evolution step 14: f(x)= 1.08397e-39
differential_evolution step 15: f(x)= 1.08397e-39
differential_evolution step 16: f(x)= 3.28457e-55
differential_evolution step 17: f(x)= 3.28457e-55
differential_evolution step 18: f(x)= 2.04655e-56
differential_evolution step 19: f(x)= 1.15437e-56
differential_evolution step 20: f(x)= 3.39189e-58
different

### Função Exponencial

In [91]:
bounds = [(0.0, 10.0), (pow(10.0, exp=-6), 10.0), (pow(10.0, exp=-6), 10.0), (0.0, 1.0)]
result = differential_evolution(exponential_function, bounds = bounds, popsize = 40, maxiter = 30, recombination = 0.3, disp = True, mutation=0.5, seed=1)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution_exponential = result['x']
evaluation = exponential_function(solution_exponential)
print('Solution_exponential: f(%s) = %.5f' % (solution_exponential, evaluation))
print(result.fun)

differential_evolution step 1: f(x)= 5.15961e-12
differential_evolution step 2: f(x)= 5.15961e-12
differential_evolution step 3: f(x)= 2.87923e-12
differential_evolution step 4: f(x)= 2.87923e-12
differential_evolution step 5: f(x)= 2.87923e-12
differential_evolution step 6: f(x)= 1.84756e-12
differential_evolution step 7: f(x)= 1.37163e-12
differential_evolution step 8: f(x)= 1.0752e-12
differential_evolution step 9: f(x)= 1.0752e-12
differential_evolution step 10: f(x)= 1.0752e-12
differential_evolution step 11: f(x)= 1.0752e-12
differential_evolution step 12: f(x)= 1.0752e-12
differential_evolution step 13: f(x)= 9.39575e-13
differential_evolution step 14: f(x)= 9.39575e-13
differential_evolution step 15: f(x)= 9.39575e-13
differential_evolution step 16: f(x)= 9.39575e-13
differential_evolution step 17: f(x)= 4.39207e-14
differential_evolution step 18: f(x)= 4.39207e-14
differential_evolution step 19: f(x)= 4.08319e-14
differential_evolution step 20: f(x)= 4.08319e-14
differential_e

### Função Logarítmica

In [92]:
bounds = [(0.0, 10.0), (pow(10.0, exp=-6), 10.0), (0.0, 1.0)]
result = differential_evolution(logarithmic_function, bounds = bounds, popsize = 40, maxiter = 30, recombination = 0.3, disp = True, mutation=0.5, seed=1)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution_logarithmic = result['x']
evaluation = logarithmic_function(solution_logarithmic)
print('Solution_logarithmic: f(%s) = %.5f' % (solution_logarithmic, evaluation))
print(result.fun)

differential_evolution step 1: f(x)= 2.15944e-06
differential_evolution step 2: f(x)= 2.15944e-06
differential_evolution step 3: f(x)= 2.15944e-06
differential_evolution step 4: f(x)= 2.15944e-06
differential_evolution step 5: f(x)= 2.09685e-06
differential_evolution step 6: f(x)= 2.09685e-06
differential_evolution step 7: f(x)= 2.09685e-06
differential_evolution step 8: f(x)= 2.09685e-06
differential_evolution step 9: f(x)= 2.09685e-06
differential_evolution step 10: f(x)= 2.09685e-06
differential_evolution step 11: f(x)= 2.09685e-06
differential_evolution step 12: f(x)= 2.09685e-06
differential_evolution step 13: f(x)= 2.09685e-06
differential_evolution step 14: f(x)= 1.81066e-06
differential_evolution step 15: f(x)= 1.81066e-06
differential_evolution step 16: f(x)= 1.81066e-06
differential_evolution step 17: f(x)= 1.81066e-06
differential_evolution step 18: f(x)= 1.81066e-06
differential_evolution step 19: f(x)= 1.81066e-06
differential_evolution step 20: f(x)= 1.81066e-06
different

### Função Hiperbólica

In [93]:
bounds = [(0.0, 10.0), (pow(10.0, exp=-6), 10.0), (pow(10.0, exp=-6), 10.0), (0.0, 1.0)]
result = differential_evolution(hyperbolic_function, bounds = bounds, popsize = 40, maxiter = 30, recombination = 0.3, disp = True, mutation=0.5, seed=1)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution_hyperbolic = result['x']
evaluation = hyperbolic_function(solution_hyperbolic)
print('Solution_hyperbolic: f(%s) = %.5f' % (solution_hyperbolic, evaluation))
print(result.fun)

differential_evolution step 1: f(x)= 6.30882e-06
differential_evolution step 2: f(x)= 6.30882e-06
differential_evolution step 3: f(x)= 6.30882e-06
differential_evolution step 4: f(x)= 6.30882e-06
differential_evolution step 5: f(x)= 5.84139e-06
differential_evolution step 6: f(x)= 5.84139e-06
differential_evolution step 7: f(x)= 5.84139e-06
differential_evolution step 8: f(x)= 5.84139e-06
differential_evolution step 9: f(x)= 5.84139e-06
differential_evolution step 10: f(x)= 5.84139e-06
differential_evolution step 11: f(x)= 5.47638e-06
differential_evolution step 12: f(x)= 5.47638e-06
differential_evolution step 13: f(x)= 5.47638e-06
differential_evolution step 14: f(x)= 5.47638e-06
differential_evolution step 15: f(x)= 4.40167e-06
differential_evolution step 16: f(x)= 4.40167e-06
differential_evolution step 17: f(x)= 4.40167e-06
differential_evolution step 18: f(x)= 4.40167e-06
differential_evolution step 19: f(x)= 4.40167e-06
differential_evolution step 20: f(x)= 4.40167e-06
different

### Função de Curva em S

In [94]:
bounds = [(0.0, 10.0), (pow(10.0, exp=-6), 10.0), (0.0, 1.0)]
result = differential_evolution(S_curve_function, bounds = bounds, popsize = 40, maxiter = 30, recombination = 0.3, disp = True, mutation=0.5, seed=1)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])
# evaluate solution
solution_S_curve = result['x']
evaluation = S_curve_function(solution_S_curve)
print('Solution_S_curve: f(%s) = %.5f' % (solution_S_curve, evaluation))
print(result.fun)

differential_evolution step 1: f(x)= 3.28687e-06
differential_evolution step 2: f(x)= 3.28687e-06
differential_evolution step 3: f(x)= 3.28687e-06
differential_evolution step 4: f(x)= 3.28687e-06
differential_evolution step 5: f(x)= 3.28687e-06
differential_evolution step 6: f(x)= 1.68133e-06
differential_evolution step 7: f(x)= 1.68133e-06
differential_evolution step 8: f(x)= 1.68133e-06
differential_evolution step 9: f(x)= 1.68133e-06
differential_evolution step 10: f(x)= 8.93472e-07
differential_evolution step 11: f(x)= 8.93472e-07
differential_evolution step 12: f(x)= 8.93472e-07
differential_evolution step 13: f(x)= 8.93472e-07
differential_evolution step 14: f(x)= 8.93472e-07
differential_evolution step 15: f(x)= 8.93472e-07
differential_evolution step 16: f(x)= 8.93472e-07
differential_evolution step 17: f(x)= 6.96932e-07
differential_evolution step 18: f(x)= 6.96932e-07
differential_evolution step 19: f(x)= 6.96932e-07
differential_evolution step 20: f(x)= 3.03774e-08
different

## Testando Otimização de parâmetros

In [95]:
power_weights_LR_optimized = []
exponential_weights_LR_optimized = []
logarithmic_weights_LR_optimized = []
hyperbolic_weights_LR_optimized = []
S_curve_weights_LR_optimized = []

for w in init_weights:
    power_w = power_function(solution_power)
    power_weights_LR_optimized.append(power_w)

    exponential_w = exponential_function(solution_exponential)
    exponential_weights_LR_optimized.append(exponential_w)

    logarithmic_w = logarithmic_function(solution_logarithmic)
    logarithmic_weights_LR_optimized.append(logarithmic_w)

    hyperbolic_w = hyperbolic_function(solution_hyperbolic)
    hyperbolic_weights_LR_optimized.append(hyperbolic_w)

    S_curve_w = S_curve_function(solution_S_curve)
    S_curve_weights_LR_optimized.append(S_curve_w)

In [96]:
drebin_power_optimized = pd.DataFrame()
drebin_exponential_optimized = pd.DataFrame()
drebin_logarithmic_optimized = pd.DataFrame()
drebin_hyperbolic_optimized = pd.DataFrame()
drebin_S_curve_optimized = pd.DataFrame()

### Função de Potência

In [97]:
count = 0
count_aux_power_optimized = []
for weight in power_weights_LR_optimized:
    if weight >= 0 and weight <= max:
        count_aux_power_optimized.append(count)
    else:
        print('')
    count += 1

In [98]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_power_optimized:
        if count == x:
            drebin_power_optimized[col] = drebin[col]
            drebin_power_optimized.replace(1, power_weights_LR_optimized[x], inplace=True)
    count+=1

### Função Exponencial

In [99]:
count = 0
count_aux_exponential_optimized = []
for weight in exponential_weights_LR_optimized:
    if weight >= 0 and weight <= max:
        count_aux_exponential_optimized.append(count)
    else:
        ''
    count += 1

In [100]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_exponential_optimized:
        if count == x:
            drebin_exponential_optimized[col] = drebin[col]
            drebin_exponential_optimized.replace(1, exponential_weights_LR_optimized[x], inplace=True)
    count+=1

### Função Logarítmica

In [101]:
count = 0
count_aux_logarithmic_optimized = []
for weight in logarithmic_weights_LR_optimized:
    if weight >= 0 and weight <= max:
        count_aux_logarithmic_optimized.append(count)
    else:
        ''
    count += 1

In [102]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_logarithmic_optimized:
        if count == x:
            drebin_logarithmic_optimized[col] = drebin[col]
            drebin_logarithmic_optimized.replace(1, logarithmic_weights_LR_optimized[x], inplace=True)
    count+=1

### Função Hiperbólica

In [103]:
count = 0
count_aux_hyperbolic_optimized = []
for weight in hyperbolic_weights_LR_optimized:
    if weight >= 0 and weight <= max:
        count_aux_hyperbolic_optimized.append(count)
    else:
        ''
    count += 1

In [104]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_hyperbolic_optimized:
        if count == x:
            drebin_hyperbolic_optimized[col] = drebin[col]
            drebin_hyperbolic_optimized.replace(1, hyperbolic_weights_LR_optimized[x], inplace=True)
    count+=1

### Função de Forma de S em Curva

In [105]:
count = 0
count_aux_S_curve_optimized = []
for weight in S_curve_weights_LR_optimized:
    if weight >= 0 and weight <= max:
        count_aux_S_curve_optimized.append(count)
    else:
        ''
    count += 1

In [106]:
count = 0
for col in drebin.drop(columns=['class']):
    for x in count_aux_S_curve_optimized:
        if count == x:
            drebin_S_curve_optimized[col] = drebin[col]
            drebin_S_curve_optimized.replace(1, S_curve_weights_LR_optimized[x], inplace=True)
    count+=1

### Shapes

In [107]:
print("Drebin >>", drebin.shape)
print("\nDrebin_Power >>", drebin_power_optimized.shape)
print("Drebin_Exponential >>", drebin_exponential_optimized.shape)
print("Drebin_Logarithmic >>", drebin_logarithmic_optimized.shape)
print("Drebin_Hyperbolic >>", drebin_hyperbolic_optimized.shape)
print("Drebin_S_Curve >>", drebin_S_curve_optimized.shape)

Drebin >> (15036, 215)

Drebin_Power >> (15036, 28)
Drebin_Exponential >> (15036, 28)
Drebin_Logarithmic >> (15036, 28)
Drebin_Hyperbolic >> (15036, 28)
Drebin_S_Curve >> (15036, 28)


In [108]:
drebin_power_optimized['class'] = drebin['class']
drebin_exponential_optimized['class'] = drebin['class']
drebin_logarithmic_optimized['class'] = drebin['class']
drebin_hyperbolic_optimized['class'] = drebin['class']
drebin_S_curve_optimized['class'] = drebin['class']

#### Treinando modelos com features selecionadas

Variáveis preditoras e alvo

In [109]:
# Power
x_power_optimized = drebin_power_optimized.drop(columns=['class'])
y_power_optimized = drebin_power_optimized['class']

# Exponential
x_exponential_optimized = drebin_exponential_optimized.drop(columns=['class'])
y_exponential_optimized = drebin_exponential_optimized['class']

# Logarithmic
x_logarithmic_optimized = drebin_logarithmic_optimized.drop(columns=['class'])
y_logarithmic_optimized = drebin_logarithmic_optimized['class']

# Hyperbolic
x_hyperbolic_optimized = drebin_hyperbolic_optimized.drop(columns=['class'])
y_hyperbolic_optimized = drebin_hyperbolic_optimized['class']

# S_Curve
x_S_curve_optimized = drebin_S_curve_optimized.drop(columns=['class'])
y_S_curve_optimized = drebin_S_curve_optimized['class']

Variáveis de treino e teste

In [110]:
x_train_power_optimized, x_test_power_optimized, y_train_power_optimized, y_test_power_optimized = train_test_split(x_power_optimized, y_power_optimized, train_size=0.8)
x_train_exponential_optimized, x_test_exponential_optimized, y_train_exponential_optimized, y_test_exponential_optimized = train_test_split(x_exponential_optimized, y_exponential_optimized, train_size=0.8)
x_train_logarithmic_optimized, x_test_logarithmic_optimized, y_train_logarithmic_optimized, y_test_logarithmic_optimized = train_test_split(x_logarithmic_optimized, y_logarithmic_optimized, train_size=0.8)
x_train_hyperbolic_optimized, x_test_hyperbolic_optimized, y_train_hyperbolic_optimized, y_test_hyperbolic_optimized = train_test_split(x_hyperbolic_optimized, y_hyperbolic_optimized, train_size=0.8)
x_train_S_curve_optimized, x_test_S_curve_optimized, y_train_S_curve_optimized, y_test_S_curve_optimized = train_test_split(x_S_curve_optimized, y_S_curve_optimized, train_size=0.8)

#### Modelos LR

In [111]:
model_LR_power_optimized = LogisticRegression()
model_LR_exponential_optimized = LogisticRegression()
model_LR_logarithmic_optimized = LogisticRegression()
model_LR_hyperbolic_optimized = LogisticRegression()
model_LR_S_curve_optimized = LogisticRegression()

Treinamento dos modelos

In [112]:
model_LR_power_optimized.fit(x_train_power_optimized, y_train_power_optimized)
model_LR_exponential_optimized.fit(x_train_exponential_optimized, y_train_exponential_optimized)
model_LR_logarithmic_optimized.fit(x_train_logarithmic_optimized, y_train_logarithmic_optimized)
model_LR_hyperbolic_optimized.fit(x_train_hyperbolic_optimized, y_train_hyperbolic_optimized)
model_LR_S_curve_optimized.fit(x_train_S_curve_optimized, y_train_S_curve_optimized)

LogisticRegression()

#### Métricas

In [113]:
# Power
print("Acurácia >> ", (cross_val_score(model_LR_power_optimized, x_test_power_optimized, y_test_power_optimized, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_LR_power_optimized, x_test_power_optimized, y_test_power_optimized, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_LR_power_optimized, x_test_power_optimized, y_test_power_optimized, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_LR_power_optimized, x_test_power_optimized, y_test_power_optimized, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  62.30054007440533 %
Precisão >>  0.0 %


/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: Un

Recall >>  0.0 %
F1-Score >>  0.0 %


In [114]:
# Exponential
print("Acurácia >> ", (cross_val_score(model_LR_exponential_optimized, x_test_exponential_optimized, y_test_exponential_optimized, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_LR_exponential_optimized, x_test_exponential_optimized, y_test_exponential_optimized, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_LR_exponential_optimized, x_test_exponential_optimized, y_test_exponential_optimized, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_LR_exponential_optimized, x_test_exponential_optimized, y_test_exponential_optimized, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  63.92949734937894 %
Precisão >>  0.0 %
Recall >>  0.0 %


/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: Un

F1-Score >>  0.0 %


In [115]:
# Logarithmic
print("Acurácia >> ", (cross_val_score(model_LR_logarithmic_optimized, x_test_logarithmic_optimized, y_test_logarithmic_optimized, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_LR_logarithmic_optimized, x_test_logarithmic_optimized, y_test_logarithmic_optimized, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_LR_logarithmic_optimized, x_test_logarithmic_optimized, y_test_logarithmic_optimized, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_LR_logarithmic_optimized, x_test_logarithmic_optimized, y_test_logarithmic_optimized, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  63.49738254625458 %
Precisão >>  0.0 %


/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: Un

Recall >>  0.0 %
F1-Score >>  0.0 %


In [116]:
# Hyperbolic
print("Acurácia >> ", (cross_val_score(model_LR_hyperbolic_optimized, x_test_hyperbolic_optimized, y_test_hyperbolic_optimized, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_LR_hyperbolic_optimized, x_test_hyperbolic_optimized, y_test_hyperbolic_optimized, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_LR_hyperbolic_optimized, x_test_hyperbolic_optimized, y_test_hyperbolic_optimized, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_LR_hyperbolic_optimized, x_test_hyperbolic_optimized, y_test_hyperbolic_optimized, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  62.53326405050276 %
Precisão >>  0.0 %
Recall >>  0.0 %


/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: Un

F1-Score >>  0.0 %


In [117]:
# S_curve
print("Acurácia >> ", (cross_val_score(model_LR_S_curve_optimized, x_test_S_curve_optimized, y_test_S_curve_optimized, scoring="accuracy", cv = 5).mean())*100,"%")
print("Precisão >> ", (cross_val_score(model_LR_S_curve_optimized, x_test_S_curve_optimized, y_test_S_curve_optimized, scoring="precision", cv = 5).mean())*100,"%")
print("Recall >> ", (cross_val_score(model_LR_S_curve_optimized, x_test_S_curve_optimized, y_test_S_curve_optimized, scoring="recall", cv = 5).mean())*100,"%")
print("F1-Score >> ", (cross_val_score(model_LR_S_curve_optimized, x_test_S_curve_optimized, y_test_S_curve_optimized, scoring="f1", cv = 5).mean())*100,"%")

Acurácia >>  62.4002631273459 %
Precisão >>  0.0 %


/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/overycall/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: Un

Recall >>  0.0 %
F1-Score >>  0.0 %
